In [ ]:
#!pip install --upgrade pip setuptools wheel
#!python -m pip install opencv-python
#!pip install matplotlib
#!pip freeze
#!pip install opencv-python-headless
#!pip install scikit-image
#!pip install -U scikit-fuzzy
#!pip install pandas
#!pip install scikit-learn
#!pip install tqdm==4.40.0
#!pip3 install opencv-contrib-python

import os
import cv2
import library
import numpy as np
import hair_removal
import pandas as pd
import glcm_features
import color_features
import texture_features
from os.path import join
from matplotlib import pyplot as plt

### GENERATING FEATURES

In [ ]:
flag = True

# path to the output folder to save the features created
output = os.path.join('/home',
                     'emily',
                     'Desktop',
                     'CAD',
                     'MelanomaChallenge',
                     'features')

# get a random sample of the path
samples, flag = library.get_sample("/home/emily/Desktop/CAD/challenge1/train",
                                   os.path.join(output,'featuresCh1TestD.csv'),# name of the csv file in the output folder
                                   flag)

# initial variables
dictF = {}
features = pd.DataFrame()
count = 0

for sample in samples:
    count += 1
    
    # read image
    img = cv2.imread(sample)
    
    # clahe preprocessing
    clahe = library.clahe_rgb(img, 8)
    
    # white balance
    white_balance = cv2_white_balance(clahe)
    
    # hair removal
    white_balance = white_balance.astype("uint8")
    hairless = hair_removal.hair_remove(white_balance, 17, 4)
    
    dictF['name'] = sample
    dictF['label'] = (0 if 'nevus' in sample else 1 )
    
    # color features
    colors = color_features.extract_color_features(hairless)
    dictF.update(colors)
    
    # glcm features
    angles = [0, np.pi/4, np.pi/2, 3*np.pi/4]
    distances = [1]
    colorspaces = ['rgb', 'hsv', 'lab', 'ycc', 'gray']
    
    for cs in colorspaces:
        glcm = glcm_features.get_glcm(hairless, angles, distances, cs)
        dictF.update(glcm)
    
    
    # lbp features
    lbp = texture_features.extract_lbp(hairless, 1, 8)
    dictF.update(lbp)
    
    #zernike features
    hairless = cv2.cvtColor(clean, cv2.COLOR_RGB2GRAY)
    zernike = shape_features.extract_zernike_moments(hairless[th], 200)
    dictF.update(zernike)
        
    # orb features
    hairless = cv2.cvtColor(hairless, cv2.COLOR_BGR2RGB)
    hairless = np.uint16(hairless)
    orb = texture_features.extract_orb(hairless, 64)
    dictF.update(orb)
    
    features = features.append(dictF, ignore_index=True)
    
    # save features
    library.writeFeatures(features,
                  flag,
                  output,
                  'featuresCh1TestD.csv')
    
    # free in memory
    flag = False
    del img
    del clahe
    del hairless
    del mask
    del colors
    del features
    
    # restarting stuff
    flag = False
    features = pd.DataFrame()
    dictF.clear()
